In [2]:
import cv2
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from skimage.feature import hog
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report
from skimage import data, exposure
from sklearn import datasets, svm, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances

In [3]:
#
# Color histogram as Global Feature
#
def compute_color_histogram(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    hist_h = cv2.calcHist([hsv], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([hsv], [1], None, [256], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], None, [256], [0, 256])
    hist = np.vstack((hist_h, hist_s, hist_v)).flatten()
    return hist

    # plt.figure(figsize=(10,5))
    # plt.title("Color Histogram (HSV)")
    # plt.xlabel("Bins")
    # plt.ylabel("# of Pixels")
    # plt.plot(hist_h, color='r', label='Hue')
    # plt.plot(hist_s, color='g', label='Saturation')
    # plt.plot(hist_v, color='b', label='Value')
    # plt.legend()
    # plt.xlim([0, 256])
    # plt.show()

#
# SIFT keypoints as Local Feature
#
def compute_sift_features(image, HPs={'nfeatures':200, 'sigma':1.6, 'contrastThreshold':0.04, 'edgeThreshold':10}):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create(nfeatures=HPs['nfeatures'])
    sift.setContrastThreshold(HPs['contrastThreshold'])
    sift.setEdgeThreshold(HPs['edgeThreshold'])
    sift.setSigma(HPs['sigma'])
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    # print(f"Number of keypoints: {len(keypoints)}")
    # print(descriptors.shape)
    
    keypoints = keypoints[:HPs['nfeatures']]
    descriptors = descriptors[:HPs['nfeatures']]
    sift_image = cv2.drawKeypoints(image, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    
    # Display the image with keypoints
    # plt.figure(figsize=(10,10))
    # plt.title(f"SIFT Keypoints: {len(keypoints)} detected")
    # plt.imshow(cv2.cvtColor(sift_image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    return descriptors

def main():
    image_path = 'C:\\source\\repos\\cv-assignment\\5-Categories\\buddha\\image_0001.jpg'
    image = cv2.imread(image_path)

    # plt.figure(figsize=(10,10))
    # plt.title("Original Image")
    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # plt.axis('off')
    # plt.show()
    
    compute_color_histogram(image)
    # compute_sift_features(image)

if __name__ == "__main__":
    main()

In [4]:
def ComputeXY(train_folder, allowed_sports, Sift_HPs):
    X = np.array([])
    y = np.array([])
    for folder_name in os.listdir(train_folder):
        if folder_name not in allowed_sports: continue
        folder_dir = os.path.join(train_folder, folder_name)
        print(f"Reading images from folder: {folder_name}")
        for image_name in os.listdir(folder_dir):
            image_path = os.path.join(folder_dir, image_name)
            if os.path.isfile(image_path):
                image = cv2.imread(image_path)
                #
                histo_X = compute_color_histogram(image)
                sift_X = compute_sift_features(image, HPs=Sift_HPs).flatten() # returns a 2D array of descriptors
                max_len = 128 * Sift_HPs['nfeatures'] # TODO MAGIC NUM 128
                if len(sift_X) < max_len:
                    padding = np.zeros(max_len - len(sift_X))
                    sift_X = np.concatenate((sift_X, padding))
                X_one = np.concatenate((sift_X, histo_X))
                #
                X = X_one if X.size == 0 else np.vstack((X, X_one))
                #
                one_hot_encoding = np.zeros(len(allowed_sports))
                one_hot_encoding[allowed_sports.index(folder_name)] = 1
                y = one_hot_encoding if y.size == 0 else np.vstack((y, one_hot_encoding))
    return X, y
    

In [11]:
TRAIN_FOLDER = 'C:\\Users\\sseksaria\\Downloads\\SportsData\\train'
ALLOWED_SPORTS = ['air hockey', 'ampute football', 'archery']#, 'arm wrestling', 'axe throwing', 'balance beam', 'barell racing', 'baseball', 'basketball', 'baton twirling']
Sift_HPs = [
            # {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 3},
            {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 2},
            {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 1.6},
            {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 1.2},
            # {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 1}
            ]


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim

for HP in Sift_HPs:
    print(f'\n\n\n############### Sift HP: {HP} ###############')
    X, y = ComputeXY(TRAIN_FOLDER, ALLOWED_SPORTS, HP)
    print(f'X: {X.shape}')
    print(f'y: {y.shape}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert numpy arrays to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Define the model architecture
    model = nn.Sequential(
        nn.Linear(X_train.shape[1], 512),
        nn.ReLU(),
        nn.Linear(512, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, len(ALLOWED_SPORTS)),
        nn.Softmax(dim=1)
    )

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    batch_size = 16
    num_batches = len(X_train) // batch_size

    for epoch in range(100):
        for batch in range(num_batches):
            start = batch * batch_size
            end = start + batch_size

            optimizer.zero_grad()
            outputs = model(X_train_tensor[start:end])
            loss = criterion(outputs, torch.argmax(y_train_tensor[start:end], dim=1))
            loss.backward()
            optimizer.step()

    with torch.no_grad():
        outputs = model(X_test_tensor)
        _, predicted = torch.max(outputs, dim=1)
        accuracy = torch.sum(predicted == torch.argmax(y_test_tensor, dim=1)).item() / len(y_test)
        print("Accuracy:", accuracy)
        classification_rep = metrics.classification_report(y_test_tensor.argmax(dim=1).numpy(), predicted.numpy())
        print("Classification Report:")
        print(classification_rep)




############### Sift HP: {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 2} ###############
Reading images from folder: air hockey
Reading images from folder: ampute football
Reading images from folder: archery
X: (356, 26368)
y: (356, 3)
Accuracy: 0.375
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        26
           1       0.00      0.00      0.00        19
           2       0.38      1.00      0.55        27

    accuracy                           0.38        72
   macro avg       0.12      0.33      0.18        72
weighted avg       0.14      0.38      0.20        72




############### Sift HP: {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 1.6} ###############
Reading images from folder: air hockey


c:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

Reading images from folder: ampute football
Reading images from folder: archery
X: (356, 26368)
y: (356, 3)
Accuracy: 0.3611111111111111
Classification Report:
              precision    recall  f1-score   support

           0       0.36      1.00      0.53        26
           1       0.00      0.00      0.00        19
           2       0.00      0.00      0.00        27

    accuracy                           0.36        72
   macro avg       0.12      0.33      0.18        72
weighted avg       0.13      0.36      0.19        72




############### Sift HP: {'nfeatures': 200, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 1.2} ###############
Reading images from folder: air hockey


c:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\sseksaria\.conda\envs\py311-cv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

Reading images from folder: ampute football
Reading images from folder: archery
X: (356, 26368)
y: (356, 3)


KeyboardInterrupt: 